<a href="https://colab.research.google.com/github/28Aarya/FHI-score-for-stability/blob/main/Financial_Health_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
pip install yfinance

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [50]:
# Fetch the stocks and relevant data
import yfinance as yf
import pandas as pd

stocks = ['AAPL', 'MSFT', 'NVDA', 'NOW', 'NOK', 'INTC', 'TSLA', 'ZM']

def fetch_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    # Define metrics inside the function
    metrics = {
        'ticker': ticker,
        'currentRatio': info.get('currentRatio', None),
        'debtToEquity': info.get('debtToEquity', None),
        'returnOnEquity': info.get('returnOnEquity', None),
        'priceToBook': info.get('priceToBook', None),
        'earningsGrowth': info.get('earningsGrowth', None),
        'dividendYield': info.get('dividendYield', None), #dividend payout replace
        'beta': info.get('beta', None)
    }

    return metrics

financial_data = [fetch_metrics(stock) for stock in stocks]
financial_df = pd.DataFrame(financial_data)
financial_df.head(8)


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.022034,-0.341,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.777143,0.104,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,81.393350,1.680,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.554232,0.769,NaN,0.986
4,NOK,1.676,22.538,0.04488,1.119743,0.506,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.060652,NaN,0.0224,1.026
6,TSLA,1.844,18.078,0.20389,16.168028,0.170,NaN,2.295
7,ZM,4.559,0.751,0.11275,3.102601,0.173,NaN,-0.055


In [52]:
financial_df.isnull().sum()

,0
ticker,0
currentRatio,0
debtToEquity,0
returnOnEquity,0
priceToBook,0
earningsGrowth,1
dividendYield,3
beta,0


In [53]:
financial_df['dividendYield'].fillna(00, inplace=True) #cause company doesnt give out dividends to share holders
financial_df.head(8)

<ipython-input-53-71849bc3da29>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  financial_df['dividendYield'].fillna(00, inplace=True) #cause company doesnt give out dividends to share holders


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.022034,-0.341,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.777143,0.104,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,81.393350,1.680,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.554232,0.769,0.0000,0.986
4,NOK,1.676,22.538,0.04488,1.119743,0.506,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.060652,NaN,0.0224,1.026
6,TSLA,1.844,18.078,0.20389,16.168028,0.170,0.0000,2.295
7,ZM,4.559,0.751,0.11275,3.102601,0.173,0.0000,-0.055


In [54]:
# Earnings per share for 2023 and 2024 as per the internet for INTC
eps_2024 = -0.46
eps_2023 = 0.54

# Calculate earnings growth
earnings_growth = (eps_2024 - eps_2023) / eps_2023
financial_df['earningsGrowth'].fillna(earnings_growth, inplace=True)
financial_df.head(6)

<ipython-input-54-25fec0f18ac4>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  financial_df['earningsGrowth'].fillna(earnings_growth, inplace=True)


,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta
0,AAPL,0.867,209.059,1.57413,61.022034,-0.341000,0.0044,1.240
1,MSFT,1.301,33.657,0.35604,10.777143,0.104000,0.0080,0.904
2,NVDA,4.269,17.221,1.23767,81.393350,1.680000,0.0003,1.657
3,NOW,1.134,24.166,0.16221,23.554232,0.769000,0.0000,0.986
4,NOK,1.676,22.538,0.04488,1.119743,0.506000,0.0337,0.737
5,INTC,1.312,47.906,-0.15597,1.060652,-1.851852,0.0224,1.026


In [60]:
#standardise the data to make it more comparable

std_columns= ['currentRatio',	'debtToEquity',	'returnOnEquity',	'priceToBook',	'earningsGrowth',	'dividendYield',	'beta'] #dividend payout ratio instead
financial_df[std_columns] = financial_df[std_columns].apply(lambda x: (x - x.mean()) / x.std())
financial_df.head(6)

,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta,FHI,Performance
0,AAPL,-0.864646,2.423681,1.829481,1.200397,-0.488585,-0.330681,0.206331,58.926869,Overperforming
1,MSFT,-0.565220,-0.194253,-0.138821,-0.463556,-0.046803,-0.047240,-0.284481,9.944822,Underperforming
2,NVDA,1.482473,-0.439566,1.285798,1.875031,1.517803,-0.653488,0.815464,14.010249,Underperforming
3,NOW,-0.680437,-0.335910,-0.452030,-0.040419,0.613390,-0.677108,-0.164700,8.839065,Underperforming
4,NOK,-0.306498,-0.360208,-0.641622,-0.783379,0.352291,1.976212,-0.528427,6.265270,Underperforming
5,INTC,-0.557630,0.018418,-0.966174,-0.785336,-1.988514,1.086523,-0.106270,12.247726,Underperforming


In [61]:
financial_df.describe()

,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta,FHI
count,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000
mean,4.163336e-17,0.000000,-5.551115e-17,-5.551115e-17,0.000000,5.551115e-17,1.942890e-16,14.834837
std,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,18.216981
min,-8.646464e-01,-0.685387,-9.661742e-01,-7.853356e-01,-1.988514,-6.771084e-01,-1.685341e+00,1.674860
25%,-5.940239e-01,-0.429973,-5.593693e-01,-7.341292e-01,-0.157248,-6.771084e-01,-3.454676e-01,6.643691
50%,-4.320645e-01,-0.348059,-4.183545e-01,-3.742913e-01,0.020209,-4.920846e-01,-1.354846e-01,9.391944
75%,2.276747e-01,-0.141086,2.173335e-01,2.697852e-01,0.417566,2.362006e-01,3.586143e-01,12.688357
max,1.682550e+00,2.423681,1.829481e+00,1.875031e+00,1.517803,1.976212e+00,1.747423e+00,58.926869


In [62]:
# Assign weights ficused on FHI for stability

weights = {
    'currentRatio': 0.25,
    'debtToEquity': 0.25,
    'returnOnEquity': 0.2,
    'priceToBook': 0.1,
    'earningsGrowth': 0.1,
    'dividendYield': 0.1,
    'beta': 0.05
}
#reasign them considering the metric replacement

In [57]:
# Function to calculate FHI for each stock
def calculate_fhi(row, weights):
    fhi_score = 0
    for metric, weight in weights.items():
        if pd.notnull(row[metric]):
            fhi_score += row[metric] * weight
    return fhi_score

# Apply the function to calculate FHI
financial_df['FHI'] = financial_df.apply(calculate_fhi, axis=1, weights=weights)
print(financial_df[['ticker', 'FHI']])


  ticker        FHI
0   AAPL  58.926869
1   MSFT   9.944822
2   NVDA  14.010249
3    NOW   8.839065
4    NOK   6.265270
5   INTC  12.247726
6   TSLA   6.769831
7     ZM   1.674860


In [58]:
financial_df.head(8)

,ticker,currentRatio,debtToEquity,returnOnEquity,priceToBook,earningsGrowth,dividendYield,beta,FHI
0,AAPL,0.867,209.059,1.57413,61.022034,-0.341000,0.0044,1.240,58.926869
1,MSFT,1.301,33.657,0.35604,10.777143,0.104000,0.0080,0.904,9.944822
2,NVDA,4.269,17.221,1.23767,81.393350,1.680000,0.0003,1.657,14.010249
3,NOW,1.134,24.166,0.16221,23.554232,0.769000,0.0000,0.986,8.839065
4,NOK,1.676,22.538,0.04488,1.119743,0.506000,0.0337,0.737,6.265270
5,INTC,1.312,47.906,-0.15597,1.060652,-1.851852,0.0224,1.026,12.247726
6,TSLA,1.844,18.078,0.20389,16.168028,0.170000,0.0000,2.295,6.769831
7,ZM,4.559,0.751,0.11275,3.102601,0.173000,0.0000,-0.055,1.674860


In [59]:
#calculate sector fhi, compare with the stocks for further data analysis
sector_avg_fhi = financial_df['FHI'].mean()

# Performance of stocks
financial_df['Performance'] = financial_df['FHI'].apply(lambda x: 'Overperforming' if x > sector_avg_fhi else 'Underperforming')

print(f"Sector Average FHI: {sector_avg_fhi:.2f}")
print(financial_df[['ticker', 'FHI', 'Performance']])



Sector Average FHI: 14.83
  ticker        FHI      Performance
0   AAPL  58.926869   Overperforming
1   MSFT   9.944822  Underperforming
2   NVDA  14.010249  Underperforming
3    NOW   8.839065  Underperforming
4    NOK   6.265270  Underperforming
5   INTC  12.247726  Underperforming
6   TSLA   6.769831  Underperforming
7     ZM   1.674860  Underperforming


In [ ]:
#detecting outliers for undervaluation and overvaluation- using z score(?)
'''def evaluate_valuation(row):
    if row['zscore'] <-1 and row['fhi'] > sector_avg['fhi']:
        return "Undervalued"
    elif row['zscore'] > 1 and row['fhi'] < sector_avg['fhi']:
        return "Overvalued"
    else:
        return "Fairly Valued"

financial_df['valuation'] = financial_df.apply(evaluate_valuation, axis=1)'''


In [38]:
'''new_df= financial_df[['ticker', 'FHI', 'Performance', 'Valuation(?)']]
new_df'''

,ticker,FHI,Performance
0,AAPL,0.416225,Overperforming
1,MSFT,-0.229877,Underperforming
2,NVDA,0.411984,Overperforming
3,NOW,0.033862,Overperforming
4,NOK,-0.200523,Underperforming
5,INTC,-0.481599,Underperforming
6,TSLA,0.157838,Overperforming
7,ZM,-0.107910,Underperforming


In [ ]:
#data insights: solvency, liquidity and profitability
'''def financial_summary(row):
  solvency=
  liqiudity=
  profitability=
  return solvency, liqiudity, profitability

financial_df['summary']= financial_df.apply(financial_summary, axis=1)
'''

In [ ]:
#report BASED ON VALUATION,summary and fhi/ performance- use reportlabs or fpdf

In [ ]:
'''
#investment recommendation ^
def stock_recommendation(row):
  if row['valuation']==
    return 'buy'
  if row ['valuation']==
    return 'sell'
  else:
    return 'hold'

financial_df['recommendation']= financial_df.apply(stock_recommendation, axis=1)
'''



In [ ]:
#prediction(?) with prophet